In [12]:
# Simple search enginge straight from the GenSim tutorials
import json
import glob
from gensim import corpora, models, similarities

def get_documents(corpus_loc):
    files = glob.glob(corpus_loc + "*json")
    output = []
    for filename in files:
        with open(filename) as f:
            for line in f:
                output.append(json.loads(line)['full_text'])
    return output

documents = get_documents("/Volumes/bigone/lens_corpus/files/")

In [13]:
from pprint import pprint   # pretty-printer

stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1] for text in texts]

dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/deerwester.dict')

new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())

corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus) # store to disk, for later use

In [ ]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
doc = "OPSB transportation"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
index = similarities.MatrixSimilarity(lsi[corpus]) # transform corpus to LSI space and index it
index.save('/tmp/deerwester.index')
index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')
sims = index[vec_lsi] # perform a similarity query against the corpus
sims = sorted(enumerate(sims), key=lambda item: -item[1])

In [17]:
print documents[265]


None of the four Recovery School District charter school boards that met Tuesday evening chose to transfer their academically improved schools to the control of the Orleans Parish School Board, with two putting the decision off another month.
The boards that oversee Mary D. Coghill Charter School and McDonogh City Park Academy voted outright to keep their schools under the Recovery district. The boards that govern Morris Jeff Community School and Fannie C. Williams Charter School delayed the decision until December. Schools have until Jan. 5 to decide.
Twenty charter school boards will have to make this decision by then, and they oversee 36 schools that are eligible to transfer to the control of the School Board.
In the four years since the first charter became eligible, no board has ever voted to transfer its school back to the auspices of the School Board  but this year one came close.
The New Beginnings Schools Foundation, in a rushed, confusing series of votes, approved a motion t